This script creates a table called las in the database containing the definition of the local authorities

In [ ]:
# Download the a dataset containing current clipped definitions of local authorities from ONS 
# This link may expire, but you can get it from here: http://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-full-clipped-boundaries-in-great-britain

url = "https://opendata.arcgis.com/datasets/686603e943f948acaa13fb5d2b0f1275_0.zip?outSR=%7B%22wkid%22%3A27700%2C%22latestWkid%22%3A27700%7D"

from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

response = urlopen(url, timeout = 5)
zipfile = ZipFile(BytesIO(response.read()))

with zipfile as z:
    z.extractall("raw/ons/")

In [ ]:
## Add folder one level up to system path so Python can 'see' the db_connections module
import sys
sys.path.append('..')

In [ ]:
import os
import datetime
import subprocess

from db_connections.secrets import host, username, dbname

psql_template = 'psql -p 5432 --host {} --username {} --dbname {}'
psql_command = psql_template.format(host, username, dbname)

filename =  "raw/ons/Local_Authority_Districts_December_2016_Full_Clipped_Boundaries_in_Great_Britain.shp"
tablename = "ons_las"

bash_command = "shp2pgsql -I -s 27700 {} {} | {}".format(filename, tablename, psql_command)


process = subprocess.Popen(bash_command, stdout=subprocess.PIPE, shell=True)  
output, error = process.communicate()

In [ ]:
# Create indexes. 
from db_connections.connections import runsql


# This isn't really needed because the psql command automatically creates a geographic index.
# sql = """
# CREATE INDEX las_polygon_gix ON ons_las USING GIST (geom);
# """

# runsql(sql)


sql = """
CREATE INDEX las_id_x ON ons_las USING BTREE (lad16cd);
"""
runsql(sql)

In [ ]:
import pandas as pd 

conn = get_conn()
pd.read_sql("select * from ons_las limit 5", conn)

The ordnance survey version of this data has a bunch of additional useful fields that give further informatino about each local authorty
It will be useful to also have a copy of this

The data can be obtained [here](https://www.ordnancesurvey.co.uk/opendatadownload/products.html#BDLINE).  Download it to `step1_create_addressbase_db/raw` and unzip

In [ ]:

psql_template = 'psql -p 5432 --host {} --username {} --dbname {}'
psql_command = psql_template.format(host, username, dbname)

filename =  "raw/bdline_essh_gb/Data/GB/district_borough_unitary_region.shp"
tablename = "os_las"

bash_command = "shp2pgsql -I -s 27700 {} {} | {}".format(filename, tablename, psql_command)


process = subprocess.Popen(bash_command, stdout=subprocess.PIPE, shell=True)  
output, error = process.communicate()


